In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Install transformer if you don't have it installed
# ! pip install transformers

This is a standalone notebook that can be imported into colab to run.

## Model Setup

In [3]:
model_name = "NumbersStation/nsql-350M" # <-- You can switch to other models like "NumbersStation/nsql-6B"

In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

## Setup table schema

This is a simple example of database table schema if you want to connect to your own PostgreSQL or SQlite please refer to [other notebooks](https://github.com/NumbersStationAI/NSQL/tree/main/examples).

In [5]:
table_schema = """CREATE TABLE stadium (
    stadium_id number,
    location text,
    name text,
    capacity number,
    highest number,
    lowest number,
    average number
)

CREATE TABLE singer (
    singer_id number,
    name text,
    country text,
    song_name text,
    song_release_year text,
    age number,
    is_male others
)

CREATE TABLE concert (
    concert_id number,
    concert_name text,
    theme text,
    stadium_id text,
    year text
)

CREATE TABLE singer_in_concert (
    concert_id number,
    singer_id text
)
"""

question = "What is the maximum, the average, and the minimum capacity of stadiums ?"

prompt = f"""{table_schema}

-- Using valid SQLite, answer the following questions for the tables provided above.

-- {question}

SELECT"""

## Generate SQL

In [9]:
input_ids = tokenizer(prompt, return_tensors="pt").input_ids
generated_ids = model.generate(input_ids, max_length=500)
output = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
output = 'SELECT' + output.split('SELECT')[-1]

print(output)

SELECT MAX(capacity), AVG(capacity), MIN(capacity) FROM stadium;
